In [35]:
using Pkg
Pkg.add("CategoricalArrays")
Pkg.add("RCall")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


   Installed VersionParsing ─ v1.3.0


   Installed WinReg ───────── v1.0.0
   Installed Conda ────────── v1.10.0


...

    Building RCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/846b2aab2d312fda5e7b099fc217c661e8fae27e/build.log`


Precompiling 

project...


  ✓ WinReg
  ✓ VersionParsing


  ✓ Conda
  3 dependencies successfully precompiled in 1 seconds. 356 already precompiled.


In [1]:
using AlgebraOfGraphics
using AlgebraOfGraphics: density
using CairoMakie
using DataFrames
using Arrow

set_aog_theme!()

In [2]:
ts = 1 # unit: seconds
tau = 60 # unit: seconds
data_dir = "../data"
format = "arrow"
fname = "events.JNO.ts_$(ts)s_tau_$(tau)s.$format"
output_path = "$data_dir/05_reporting/$fname"

# fname = f'events.JNO.ts_{ts}s_tau_{tau}s.{format}'

"../data/05_reporting/events.JNO.ts_1s_tau_60s.arrow"

In [4]:
function load(path :: String)
    df = path |> Arrow.Table |> DataFrame |> dropmissing

    # Iterate through each column in the DataFrame

    filter(row -> all(x -> !(x isa Number && isnan(x)), row), df)
end

begin
    path01 = output_path

    juno_df = load(path01)
end
# psp_df = Dataset("../data/psp_ids_dataset.parquet")

Row,time,tstart,tstop,d_tstart,d_tstop,d_time,duration,count,B_std,B_mean,dB_vec,index_diff,index_std,index_fluctuation,b_mag,b_n,bn_over_b,d_star,db_mag,db_over_b,db_over_b_max,fit.stat.chisqr,fit.stat.rsquared,fit.vars.amplitude,fit.vars.c,fit.vars.sigma,rotation_angle,dB_x,dB_y,dB_z,dB_lmn_x,dB_lmn_y,dB_lmn_z,k_x,k_y,k_z,Vl_x,Vl_y,Vl_z,Vn_x,Vn_y,Vn_z,radial_distance,plasma_density,plasma_temperature,model_b_r,model_b_t,model_b_n,v_x,v_y,v_z,plasma_speed,B_background_x,B_background_y,B_background_z,v_l,v_n,v_k,v_mn,L_n,L_mn,L_k,j0,j0_k,ion_inertial_length,Alfven_speed,j_Alfven,L_n_norm,L_mn_norm,L_k_norm,j0_norm,j0_k_norm
,DateTime,DateTime,DateTime,DateTime,DateTime,DateTime,Nanoseco…,UInt32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2015-01-01T00:16:30,2015-01-01T00:16:00,2015-01-01T00:17:00,2015-01-01T00:16:06,2015-01-01T00:16:32,2015-01-01T00:16:22.960,26000000000 nanoseconds,60,0.162546,1.19905,0.327567,0.273188,2.17518,1.16041,1.15315,0.657732,0.570381,-0.0286912,0.0684947,0.0593982,0.149578,0.0397683,0.926916,-0.42039,0.135921,3.66306,23.5176,-0.02,-0.17,-0.47,0.498278,0.0261717,-0.0351226,0.999136,-0.0242492,-0.0337454,0.0110667,0.26118,0.965227,-0.829037,-0.537313,0.154896,4.139,0.16894,31714.0,0.147051,-0.564473,-0.175,474.516,13.6558,0.1,474.712,0.476565,-0.71817,-0.293449,8.91447,400.713,473.771,474.628,10418.5,12340.3,12318.0,-0.0481045,-0.0481915,554.058,61.1781,1.65592,18.8041,22.2727,22.2324,-0.0290501,-0.0291026
2,2015-01-01T04:06:30,2015-01-01T04:06:00,2015-01-01T04:07:00,2015-01-01T04:06:29,2015-01-01T04:06:34,2015-01-01T04:06:29.735,5000000000 nanoseconds,60,0.249081,1.00657,0.494166,0.490938,2.85589,1.94761,1.0116,0.207155,0.204779,-0.216014,-0.00167739,0.00165815,0.070986,0.00106185,0.997055,-1.16217,0.802043,1.34502,38.4517,-0.29,-0.06,-0.62,0.686644,0.0246675,0.00339688,0.906544,-0.0474251,-0.419439,0.432547,0.0527013,0.90007,-0.886964,-0.154341,0.435285,4.145,0.18916,26410.0,0.168811,-0.663316,-0.0535,449.671,-1.13582,0.0,449.673,0.00223056,-1.02636,0.146017,194.444,398.667,407.701,405.459,1993.34,2027.3,2038.5,-0.42396,-0.421629,523.608,50.7193,1.53714,3.80692,3.87178,3.89318,-0.275811,-0.274295
3,2015-01-01T05:02:00,2015-01-01T05:01:30,2015-01-01T05:02:30,2015-01-01T05:01:41,2015-01-01T05:02:20,2015-01-01T05:01:56.535,39000000000 nanoseconds,60,0.300942,1.08025,0.646838,0.598785,2.77718,2.05655,1.08195,0.397574,0.367463,0.03649,-0.106837,0.0987448,0.162252,0.128393,0.957544,0.801533,-0.587806,5.49146,43.7686,-0.01,0.3,0.75,-0.802586,-0.033592,0.0856029,-0.999834,-0.0170398,-0.00651521,0.101524,0.369922,0.923499,0.927627,-0.370604,0.0464729,4.145,0.18748,26386.0,0.167869,-0.660466,-0.0529,449.056,-1.52511,0.0,449.059,0.0280861,-0.9541,0.310914,45.026,417.122,448.955,446.796,16267.8,17425.0,17509.3,0.0649912,0.0646786,525.949,54.4886,1.63671,30.9303,33.1306,33.2908,0.0397085,0.0395175
4,2015-01-01T07:37:00,2015-01-01T07:36:30,2015-01-01T07:37:30,2015-01-01T07:36:47,2015-01-01T07:37:30,2015-01-01T07:36:53.142,43000000000 nanoseconds,60,0.240445,1.16584,0.64792,0.555755,2.35454,1.90506,1.16654,-1.14043,-0.977618,-0.0158497,-0.0248477,0.0213003,0.128065,0.255089,0.850028,-1.70028,0.982537,26.8189,39.8777,0.6,0.23,-0.5,0.808083,0.074091,-0.0664215,0.570312,0.227862,0.789191,-0.811778,-0.218062,0.541725,-0.259082,0.965855,0.000551778,4.146,0.18387,26343.0,0.165611,-0.652784,-0.0519,447.865,-2.21648,0.0,447.87,0.1243,-1.09365,0.286592,-363.083,118.175,254.918,262.218,5081.51,11275.4,10961.5,-0.0481004,-0.0494779,531.087,59.323,1.74761,9.56812,2

In [ ]:
function bin_and_normalize!(df::DataFrame, column::Symbol, num_bins::Int)
    # Determine the column type and apply appropriate binning
    if is_time_column(df[:, column])
        # Convert time to numeric values (e.g., seconds since epoch) for uniform binning
        numeric_values = Dates.value.(df[:, column])
        max_val = maximum(numeric_values)
        min_val = minimum(numeric_values)
        bin_edges = range(min_val, max_val, length=num_bins+1)
        df[:binned] = cut(numeric_values, bin_edges)
    else
        # Directly bin numerical values
        max_val = maximum(df[:, column])
        min_val = minimum(df[:, column])
        bin_edges = range(min_val, max_val, length=num_bins+1)
        df[:binned] = cut(df[:, column], bin_edges)
    end

    # Group by the binned column
    gdf = groupby(df, :binned)

    # Normalize within each group for column `x`
    df.normalized_x = combine(gdf, :x => x -> x / sum(x) => :normalized_x)[:normalized_x]
end

In [36]:
using RCall

In [ ]:
select!(juno_df, Not(:duration))

In [58]:
R"""
source("utils.R")
x_col <- "radial_distance"
y_col <- "j0"
y_lim <- c(0, 5)
xlab <- "Radial Distance (AU)"
ylab <- expression(Log~J~(nA~m^-2))
p <- plot_binned_data(juno_df, x_col = x_col, y_col = y_col, x_bins = 8, y_bins = 32, y_lim = y_lim, log_y = TRUE)
p <- p + labs(x = xlab, y= ylab)
save_plot("new/current/current_r_dist")
"""

┌ Warning: RCall.jl: Saving 7 x 7 in image
│ Saving 7 x 7 in image
└ @ RCall /Users/zijin/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{StrSxp}
../figures/new/current/current_r_dist.pdf


In [56]:
describe(juno_df[!,:j0])

Summary Stats:
Length:         63143
Missing Count:  0
Mean:           -51.638258
Std. Deviation: 13293.433081
Minimum:        -3304016.509094
1st Quartile:   -0.217981
Median:         0.041078
3rd Quartile:   0.290138
Maximum:        412214.005090
Type:           Float64


In [45]:
R"""
juno_df
"""

RObject{VecSxp}
                    time              tstart               tstop
1    2015-01-01 00:16:30 2015-01-01 00:16:00 2015-01-01 00:17:00
2    2015-01-01 04:06:30 2015-01-01 04:06:00 2015-01-01 04:07:00
3    2015-01-01 05:02:00 2015-01-01 05:01:30 2015-01-01 05:02:30
4    2015-01-01 07:37:00 2015-01-01 07:36:30 2015-01-01 07:37:30
5    2015-01-01 09:37:30 2015-01-01 09:37:00 2015-01-01 09:38:00
6    2015-01-01 09:57:30 2015-01-01 09:57:00 2015-01-01 09:58:00
7    2015-01-01 13:36:30 2015-01-01 13:36:00 2015-01-01 13:37:00
8    2015-01-01 17:43:30 2015-01-01 17:43:00 2015-01-01 17:44:00
9    2015-01-01 18:36:30 2015-01-01 18:36:00 2015-01-01 18:37:00
10   2015-01-01 18:42:00 2015-01-01 18:41:30 2015-01-01 18:42:30
11   2015-01-01 20:45:30 2015-01-01 20:45:00 2015-01-01 20:46:00
12   2015-01-01 21:20:30 2015-01-01 21:20:00 2015-01-01 21:21:00
13   2015-01-01 22:13:30 2015-01-01 22:13:00 2015-01-01 22:14:00
14   2015-01-01 22:16:30 2015-01-01 22:16:00 2015-01-01 22:17:00
15   2015

In [39]:
@rput juno_df

Row,time,tstart,tstop,d_tstart,d_tstop,d_time,count,B_std,B_mean,dB_vec,index_diff,index_std,index_fluctuation,b_mag,b_n,bn_over_b,d_star,db_mag,db_over_b,db_over_b_max,fit.stat.chisqr,fit.stat.rsquared,fit.vars.amplitude,fit.vars.c,fit.vars.sigma,rotation_angle,dB_x,dB_y,dB_z,dB_lmn_x,dB_lmn_y,dB_lmn_z,k_x,k_y,k_z,Vl_x,Vl_y,Vl_z,Vn_x,Vn_y,Vn_z,radial_distance,plasma_density,plasma_temperature,model_b_r,model_b_t,model_b_n,v_x,v_y,v_z,plasma_speed,B_background_x,B_background_y,B_background_z,v_l,v_n,v_k,v_mn,L_n,L_mn,L_k,j0,j0_k,ion_inertial_length,Alfven_speed,j_Alfven,L_n_norm,L_mn_norm,L_k_norm,j0_norm,j0_k_norm
,DateTime,DateTime,DateTime,DateTime,DateTime,DateTime,UInt32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2015-01-01T00:16:30,2015-01-01T00:16:00,2015-01-01T00:17:00,2015-01-01T00:16:06,2015-01-01T00:16:32,2015-01-01T00:16:22.960,60,0.162546,1.19905,0.327567,0.273188,2.17518,1.16041,1.15315,0.657732,0.570381,-0.0286912,0.0684947,0.0593982,0.149578,0.0397683,0.926916,-0.42039,0.135921,3.66306,23.5176,-0.02,-0.17,-0.47,0.498278,0.0261717,-0.0351226,0.999136,-0.0242492,-0.0337454,0.0110667,0.26118,0.965227,-0.829037,-0.537313,0.154896,4.139,0.16894,31714.0,0.147051,-0.564473,-0.175,474.516,13.6558,0.1,474.712,0.476565,-0.71817,-0.293449,8.91447,400.713,473.771,474.628,10418.5,12340.3,12318.0,-0.0481045,-0.0481915,554.058,61.1781,1.65592,18.8041,22.2727,22.2324,-0.0290501,-0.0291026
2,2015-01-01T04:06:30,2015-01-01T04:06:00,2015-01-01T04:07:00,2015-01-01T04:06:29,2015-01-01T04:06:34,2015-01-01T04:06:29.735,60,0.249081,1.00657,0.494166,0.490938,2.85589,1.94761,1.0116,0.207155,0.204779,-0.216014,-0.00167739,0.00165815,0.070986,0.00106185,0.997055,-1.16217,0.802043,1.34502,38.4517,-0.29,-0.06,-0.62,0.686644,0.0246675,0.00339688,0.906544,-0.0474251,-0.419439,0.432547,0.0527013,0.90007,-0.886964,-0.154341,0.435285,4.145,0.18916,26410.0,0.168811,-0.663316,-0.0535,449.671,-1.13582,0.0,449.673,0.00223056,-1.02636,0.146017,194.444,398.667,407.701,405.459,1993.34,2027.3,2038.5,-0.42396,-0.421629,523.608,50.7193,1.53714,3.80692,3.87178,3.89318,-0.275811,-0.274295
3,2015-01-01T05:02:00,2015-01-01T05:01:30,2015-01-01T05:02:30,2015-01-01T05:01:41,2015-01-01T05:02:20,2015-01-01T05:01:56.535,60,0.300942,1.08025,0.646838,0.598785,2.77718,2.05655,1.08195,0.397574,0.367463,0.03649,-0.106837,0.0987448,0.162252,0.128393,0.957544,0.801533,-0.587806,5.49146,43.7686,-0.01,0.3,0.75,-0.802586,-0.033592,0.0856029,-0.999834,-0.0170398,-0.00651521,0.101524,0.369922,0.923499,0.927627,-0.370604,0.0464729,4.145,0.18748,26386.0,0.167869,-0.660466,-0.0529,449.056,-1.52511,0.0,449.059,0.0280861,-0.9541,0.310914,45.026,417.122,448.955,446.796,16267.8,17425.0,17509.3,0.0649912,0.0646786,525.949,54.4886,1.63671,30.9303,33.1306,33.2908,0.0397085,0.0395175
4,2015-01-01T07:37:00,2015-01-01T07:36:30,2015-01-01T07:37:30,2015-01-01T07:36:47,2015-01-01T07:37:30,2015-01-01T07:36:53.142,60,0.240445,1.16584,0.64792,0.555755,2.35454,1.90506,1.16654,-1.14043,-0.977618,-0.0158497,-0.0248477,0.0213003,0.128065,0.255089,0.850028,-1.70028,0.982537,26.8189,39.8777,0.6,0.23,-0.5,0.808083,0.074091,-0.0664215,0.570312,0.227862,0.789191,-0.811778,-0.218062,0.541725,-0.259082,0.965855,0.000551778,4.146,0.18387,26343.0,0.165611,-0.652784,-0.0519,447.865,-2.21648,0.0,447.87,0.1243,-1.09365,0.286592,-363.083,118.175,254.918,262.218,5081.51,11275.4,10961.5,-0.0481004,-0.0494779,531.087,59.323,1.74761,9.56812,21.2307,20.6397,-0.0275235,-0.0283117
5,2015-01-01T09:37:30,2015-01-01T09:37:00,2015-01-01T09:38:00,2015-01-01T09:3

In [26]:
arr=[DateTime(2020,1,1,1,0), DateTime(2020,1,2,0,0), DateTime(2020,1,2,1,0), DateTime(2020,1,1,0,0), ]

bin_edges = range(minimum(arr), maximum(arr), 2)

cut(arr, bin_edges)

MethodError: MethodError: no method matching isless(::Millisecond, ::DateTime)

Closest candidates are:
  isless(::Union{Day, Hour, Microsecond, Millisecond, Minute, Nanosecond, Second, Week}, !Matched::Union{Month, Quarter, Year})
   @ Dates ~/.julia/juliaup/julia-1.9.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Dates/src/periods.jl:438
  isless(::P, !Matched::P) where P<:Period
   @ Dates ~/.julia/juliaup/julia-1.9.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Dates/src/periods.jl:71
  isless(::Period, !Matched::Period)
   @ Dates ~/.julia/juliaup/julia-1.9.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.9/Dates/src/periods.jl:97
  ...


In [10]:
# bin df over time and j_0, then normalized each bin to the current time, so that the sum of each bin along j_0 at each time is 1

using DataFrames, Dates

# Sample DataFrame
df = DataFrame(time_column = [DateTime(2020,1,1,0,0), DateTime(2020,1,1,1,0), DateTime(2020,1,2,0,0), DateTime(2020,1,2,1,0)], x = [1, 2, 3, 4])

# Define your time binning function. For example, bin by day.
binning_function = x -> Dates.floor(x, Day)

# Apply the binning function to create a new column for the binned time
df.binned_time = binning_function.(df.time_column)

# Group by the binned time
gdf = groupby(df, :binned_time)

# Normalize within each group
df.normalized_x = combine(gdf, :x => (x -> x / sum(x)) => :normalized_x)[!,:normalized_x]

# Resulting DataFrame will have an additional column `normalized_x` with the normalized values


4-element Vector{Float64}:
 0.3333333333333333
 0.6666666666666666
 0.42857142857142855
 0.5714285714285714

In [12]:
df

Row,time_column,x,binned_time,normalized_x
,DateTime,Int64,DateTime,Float64
1,2020-01-01T00:00:00,1,2020-01-01T00:00:00,0.333333
2,2020-01-01T01:00:00,2,2020-01-01T00:00:00,0.666667
3,2020-01-02T00:00:00,3,2020-01-02T00:00:00,0.428571
4,2020-01-02T01:00:00,4,2020-01-02T00:00:00,0.571429


In [9]:
combine(gdf, :x => x -> x / sum(x))

Row,binned_time,x_function
,DateTime,Float64
1,2020-01-01T00:00:00,0.333333
2,2020-01-01T00:00:00,0.666667
3,2020-01-02T00:00:00,0.428571
4,2020-01-02T00:00:00,0.571429
